<a href="https://colab.research.google.com/github/sergiopegado0/Deepfakes-em-imagens/blob/main/DeepFakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Definir transformações
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Carregar dataset completo
dataset = ImageFolder(root='/content/drive/MyDrive/Datasets', transform=transform)

# Definir tamanhos dos conjuntos de treinamento e teste
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Dividir o conjunto de dados em treinamento e teste
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Criar DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f'Tamanho do conjunto de treinamento: {len(train_dataset)}')
print(f'Tamanho do conjunto de teste: {len(test_dataset)}')



Tamanho do conjunto de treinamento: 10
Tamanho do conjunto de teste: 3


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
#REDE NEURAL CNN
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: real e deepfake

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()


In [ ]:
#Treinamento
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):  # Número de épocas
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Época {epoch+1}, Perda Média: {running_loss/len(train_loader)}')

print('Treinamento concluído!')

Época 1, Perda Média: 0.687640368938446
Época 2, Perda Média: 0.6264840960502625
Época 3, Perda Média: 0.5714696049690247
Época 4, Perda Média: 0.541502058506012
Época 5, Perda Média: 0.5288042426109314
Época 6, Perda Média: 0.5199735164642334
Época 7, Perda Média: 0.5040746927261353
Época 8, Perda Média: 0.478665292263031
Época 9, Perda Média: 0.45065364241600037
Época 10, Perda Média: 0.42881888151168823
Treinamento concluído!


In [ ]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms

# Carregar imagens reais e deepfake
real_path = "/content/drive/MyDrive/Datasets/1_real"
deepfake_path = "/content/drive/MyDrive/Datasets/2_deepfake"

# Função para carregar imagens
def load_images(path):
    images = []
    for filename in os.listdir(path):
        img = Image.open(os.path.join(path, filename)).convert('RGB') # Convert to RGB
        img_tensor = transform(img)
        images.append(img_tensor)
    return images

real_images = load_images(real_path)
deepfake_images = load_images(deepfake_path)

# Comparar cada imagem real com todas as deepfake
model.eval()
correct_real = 0
total_real = len(real_images) * len(deepfake_images)

with torch.no_grad():
    for real_img in real_images:
        for fake_img in deepfake_images:
            # Remove the extra unsqueeze(0) - the tensors are already batched correctly
            # real_img = real_img.unsqueeze(0)
            # fake_img = fake_img.unsqueeze(0)
            real_output = model(real_img.unsqueeze(0)) # Add batch dimension here instead
            fake_output = model(fake_img.unsqueeze(0)) # Add batch dimension here instead
            if torch.argmax(real_output) == 0:
                correct_real += 1

print(f'Precisão na detecção de imagens reais: {100 * correct_real / total_real:.2f}%')

Precisão na detecção de imagens reais: 25.00%


In [ ]:
#verificar a precisão do modelo em detectar imagens deepfake.
correct_fake = 0
total_fake = len(real_images) * len(deepfake_images)

with torch.no_grad():
    for fake_img in deepfake_images:
        for real_img in real_images:
            # Remove the extra unsqueeze(0) - the tensors are already batched correctly
            # real_img = real_img.unsqueeze(0)  # Adicionar dimensão de batch
            # fake_img = fake_img.unsqueeze(0)  # Adicionar dimensão de batch

            # The images are already in the correct format [C, H, W]
            # We just need to add the batch dimension using unsqueeze(0)
            real_output = model(real_img.unsqueeze(0))
            fake_output = model(fake_img.unsqueeze(0))

            if torch.argmax(fake_output) == 1:
                correct_fake += 1

print(f'Precisão na detecção de imagens deepfake: {100 * correct_fake / total_fake:.2f}%')

Precisão na detecção de imagens deepfake: 90.91%
